In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [3]:
df=pd.read_csv('loan_approval_dataset.csv')
df_copy=df.copy()

In [4]:
df.drop(['loan_id'],inplace=True,axis=1)

In [5]:
encoder=OneHotEncoder()
encoded=encoder.fit_transform(df[[' education']])
df_encoded=pd.DataFrame(encoded.toarray(),columns=encoder.get_feature_names_out([' education']))
df=pd.concat([df,df_encoded],axis=1)
df.drop([' education'],axis=1,inplace=True)
encoder=OneHotEncoder()
encoded=encoder.fit_transform(df[[' self_employed']])
df_encoded=pd.DataFrame(encoded.toarray(),columns=encoder.get_feature_names_out([' self_employed']))
df=pd.concat([df,df_encoded],axis=1)
df.drop([' self_employed'],axis=1,inplace=True)


In [6]:
def fun(x):
    if 'Approved' in x:
        return '1'
    else:
        return '0'
df[' loan_status']=df[' loan_status'].apply(fun).astype(int)

In [7]:
df[' loan_status'].value_counts()

 loan_status
1    2656
0    1613
Name: count, dtype: int64

In [8]:
df[' education_ Not Graduate']=df[' education_ Not Graduate'].astype(int)
df[' education_ Graduate']=df[' education_ Graduate'].astype(int)
df[' self_employed_ No']=df[' self_employed_ No'].astype(int)
df[' self_employed_ Yes']=df[' self_employed_ Yes'].astype(int)

In [9]:
X=df.drop([' loan_status'],axis=1)
y=df[' loan_status']

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.1)
smote=SMOTE(random_state=42)
X_train,y_train=smote.fit_resample(X_train,y_train)

In [11]:
scalar=StandardScaler()
X_train=scalar.fit_transform(X_train)
X_test=scalar.transform(X_test)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(8,activation='relu'),
    Dense(4,activation='relu'),
    Dense(2,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
early_stop=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
history=model.fit(
    X_train,y_train,
    epochs=100,
    validation_data=(X_test,y_test),
    callbacks=[early_stop]
)

c:\Users\srini\PYTHON\env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.6924 - loss: 0.5975 - val_accuracy: 0.9157 - val_loss: 0.5173
Epoch 2/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9261 - loss: 0.4018 - val_accuracy: 0.9110 - val_loss: 0.4602
Epoch 3/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9430 - loss: 0.3636 - val_accuracy: 0.9204 - val_loss: 0.4184
Epoch 4/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9428 - loss: 0.3293 - val_accuracy: 0.9274 - val_loss: 0.3795
Epoch 5/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9557 - loss: 0.2891 - val_accuracy: 0.9274 - val_loss: 0.3458
Epoch 6/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9619 - loss: 0.2656 - val_accuracy: 0.9391 - val_loss: 0.3205
Epoch 7/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9630 - loss: 0.2384 - val_accuracy: 0.9415 - val_loss: 0.2880
Epoch 8/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9675 - loss: 0.2111 - val_accu

In [13]:
with open('scalar.pkl','wb') as f:
    pickle.dump(scalar,f)

In [14]:
model.save('model.h5')

In [15]:
y_test_pred=model.predict(X_test)
y_pred_class=(y_test_pred>0.5).astype(int)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [16]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred_class,y_test))

0.9672131147540983
